In [1]:
import pandas as pd
import re

# Punto 1:

Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.


In [2]:
# importiamo il dataset
loans_len=pd.read_csv('loans_lenders.csv')

In [3]:
loans_len.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [4]:
#Selezioniamo solamente l'Id del loan
loan_id=loans_len['loan_id']
loan_id

0           483693
1           483738
2           485000
3           486087
4           534428
            ...   
1387427     678999
1387428    1207353
1387429    1206220
1387430    1206425
1387431    1206486
Name: loan_id, Length: 1387432, dtype: int64

In [5]:
# con la funzione explode abbiamo separato le stringhe dei lenders ogni volta è presente la virgola attribuendole allo stesso indice
sing_lend=loans_len['lenders'].str.split(',').explode()
sing_lend

0                      muc888
0                     sam4326
0                 camaran3922
0                 lachheb1865
0                 rebecca3499
                  ...        
1387430     trogdorfamily7622
1387430             danny6470
1387430               don6118
1387431              alan5175
1387431           amy38101311
Name: lenders, Length: 28293931, dtype: object

In [6]:
# la funzione strip è servita per eliminare gli spazi bianchi tra i lender
sing_lend=sing_lend.str.strip()

In [7]:
# abbiamo creato il dataset assegnando ad ogni lender il suo loan iniziale
loan_id=pd.Series(loans_len['loan_id'])

ab={ 'loan_id': loan_id , 'lenders': sing_lend}
fin_fin=pd.DataFrame(ab)

In [8]:
fin_fin

,loan_id,lenders
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499
...,...,...
1387430,1206425,trogdorfamily7622
1387430,1206425,danny6470
1387430,1206425,don6118
1387431,1206486,alan5175


In [9]:
fin_fin = fin_fin.reset_index(drop=True)
fin_fin

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


# Punto 2: 


For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.


In [10]:
import pandas as pd
loans=pd.read_csv('loans.csv')

In [11]:
import datetime
import numpy as np

In [12]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


In [13]:
# notiamo che molte date presentano un disburse time maggiore dell'expiration time 
# crediamo che questo sia un errore in quanto la data scadenza del prestito è precedente a quella di esborso
loans.loc[loans['disburse_time'] > loans['planned_expiration_time']]

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
63,1158981,Rabiu,English,Rabiu is a Babban Gona maize farmer from Kwany...,Rabiu is a Babban Gona maize farmer from Kwany...,250.0,250.0,funded,Personal Expenses,Personal Use,...,2016-10-31 05:10:53.000 +0000,15.0,10,1,1,"#Vegan, #Repeat Borrower, #Supporting Family",male,true,bullet,field_partner
96,961161,Usman,English,Usman is a Babban Gona maize farmer. Thanks to...,Usman is a Babban Gona maize farmer. Thanks to...,200.0,200.0,funded,Personal Expenses,Personal Use,...,2015-11-12 00:39:38.000 +0000,14.0,8,2,2,#Vegan,male,true,bullet,field_partner
177,961701,Aminu,English,Aminu is a maize farmer and Babban Gona member...,Aminu is a maize farmer and Babban Gona member...,100.0,100.0,funded,Personal Expenses,Personal Use,...,2015-10-17 18:00:34.000 +0000,14.0,3,2,2,"#Parent, #Vegan",male,true,bullet,field_partner
207,1155776,Salisu,English,Salisu is a maize farmer from U/Dashire villag...,Salisu is a maize farmer from U/Dashire villag...,200.0,200.0,funded,Primary/secondary school costs,Education,...,2016-10-04 21:56:50.000 +0000,15.0,8,1,1,"#Schooling, #Vegan, #Repeat Borrower, #Parent",male,true,bullet,field_partner
217,1158968,Musa,English,Musa is a Babban Gona maize farmer from the vi...,Musa is a Babban Gona maize farmer from the vi...,125.0,125.0,funded,Personal Expenses,Personal Use,...,2016-10-03 20:56:13.000 +0000,15.0,5,1,1,"#Vegan, #Repeat Borrower",male,true,bullet,field_partner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419303,1067265,Juwyana,English,Juwayana is a Babban Gona maize farmer. She is...,Juwayana is a Babban Gona maize farmer. She is...,100.0,100.0,funded,Farming,Agriculture,...,2016-05-13 16:13:13.000 +0000,19.0,3,1,1,"#Parent, #Schooling, #Vegan",female,true,bullet,field_partner
1419304,1067985,Aliyu,English,Aliyu is a Babban Gona maize farmer from Turaw...,Aliyu is a Babban Gona maize farmer from Turaw...,350.0,350.0,funded,Farming,Agriculture,...,2016-06-11 13:46:05.000 +0000,19.0,14,1,1,"#Parent, #Schooling, #Vegan, #Sustainable Ag, ...",male,true,bullet,field_partner
1419374,1067288,Ubale,English,Ubale is a Babban Gona maize farmer. He is a h...,Ubale is a Babban Gona maize farmer. He is a h...,75.0,75.0,funded,Farming,Agriculture,...,2016-05-13 05:37:01.000 +0000,19.0,2,1,1,NaN,male,true,bullet,field_partner
1419443,1068602,Shehu,English,Shehu is a Babban Gona maize farmer from Kargi...,Shehu is a Babban Gona maize farmer from Kargi...,175.0,350.0,expired,Farming,Agriculture,...,NaN,19.0,6,1,1,"#Parent, #Schooling, #Vegan",male,true,bullet,field_partner


In [14]:
# decidiamo di invertire le date del "planned expiration time" con quelle del "disburse time" nel caso analizzato in precedenza
loans['planned_expiration_time'],loans['disburse_time']=np.where(loans['disburse_time'] > loans['planned_expiration_time'],(loans['disburse_time'],loans['planned_expiration_time']),(loans['planned_expiration_time'],loans['disburse_time']))

In [15]:
#verifichiamo che lo scambio è stato effettuato correttamente
loans.loc[loans['disburse_time'] > loans['planned_expiration_time']]

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model


In [16]:
# utilizziamo datatime per leggere le date come tali e non come stringhe
loans['planned_expiration_time'] =  pd.to_datetime(loans['planned_expiration_time'], format='%Y-%m-%d %H:%M:%S.%f')
loans['disburse_time'] =  pd.to_datetime(loans['disburse_time'], format='%Y-%m-%d %H:%M:%S.%f')

In [17]:
loans['planned_expiration_time'].head()

0   2014-02-14 03:30:06+00:00
1   2014-03-26 22:25:07+00:00
2   2014-02-15 21:10:05+00:00
3   2014-02-21 03:10:02+00:00
4   2014-02-13 06:10:02+00:00
Name: planned_expiration_time, dtype: datetime64[ns, UTC]

In [18]:
loans['disburse_time'].head()

0   2013-12-22 08:00:00+00:00
1   2013-12-20 08:00:00+00:00
2   2014-01-09 08:00:00+00:00
3   2014-01-17 08:00:00+00:00
4   2013-12-17 08:00:00+00:00
Name: disburse_time, dtype: datetime64[ns, UTC]

In [19]:
# calcoliamo la duration creando una nuova colonna
loans['duration']=loans['planned_expiration_time']-loans['disburse_time']

In [20]:
loans['duration'].head()

0   53 days 19:30:06
1   96 days 14:25:07
2   37 days 13:10:05
3   34 days 19:10:02
4   57 days 22:10:02
Name: duration, dtype: timedelta64[ns]

In [21]:
missingTime=loans[["planned_expiration_time","disburse_time", "duration"]]
missingTime.head()

,planned_expiration_time,disburse_time,duration
0,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,53 days 19:30:06
1,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,96 days 14:25:07
2,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,37 days 13:10:05
3,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,34 days 19:10:02
4,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,57 days 22:10:02


In [22]:
#verifichiamo che se una delle due date è mancante anche la duration è mancante
missingTime[missingTime['duration'].isna()]

,planned_expiration_time,disburse_time,duration
64,NaT,2010-11-18 08:00:00+00:00,NaT
65,NaT,2010-11-25 08:00:00+00:00,NaT
66,NaT,2010-11-25 08:00:00+00:00,NaT
67,NaT,2010-11-16 08:00:00+00:00,NaT
69,NaT,2010-11-10 08:00:00+00:00,NaT
...,...,...,...
1419523,NaT,2005-04-14 05:27:55+00:00,NaT
1419524,2017-02-09 23:07:54+00:00,NaT,NaT
1419525,2015-09-21 08:06:19+00:00,NaT,NaT
1419538,2017-02-25 20:21:21+00:00,NaT,NaT
